In [1]:
# import logging

# logging.basicConfig(level=logging.DEBUG)

from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json
import random

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# !wget https://huggingface.co/datasets/mesolitica/pseudolabel-malaysian-youtube-whisper-large-v3/resolve/main/pseudolabel.jsonl

In [3]:
texts = []
with open('pseudolabel.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        if l['score_ms'] < 8:
            continue
        t = l['predict_ms'][42:-13].strip()
        if len(t.split()) < 70:
            continue
        texts.append((l['audio_filename'], t))
        
        if l['score_en'] < 8:
            continue
        t = l['predict_en'][42:-13].strip()
        if len(t.split()) < 70:
            continue
        
        texts.append((l['audio_filename'], t))

2221856it [00:09, 240134.92it/s]


In [4]:
len(texts)

514126

In [5]:
from concurrent.futures import ThreadPoolExecutor, as_completed

generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=4096,
    top_p=0.95,
    top_k=50,
    repetition_penalty=1.0,
    do_sample=True,
)

In [6]:
client = InferenceClient(
    "", timeout = 40
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

def format_user(history):
    prompt = "<s>"
    for user_prompt, bot_response in history:
        prompt += f"[INST] {user_prompt} [/INST]"
        prompt += f" {bot_response}</s> "
    prompt += f"[INST]"
    return prompt

In [7]:
!mkdir mixtral-audio-instruction
# !rm mixtral-factual-wrong-v2/*.json

mkdir: cannot create directory ‘mixtral-audio-instruction’: File exists


In [8]:
texts[0]

('output-audio/1-0-0.mp3',
 'anda tahu keuntungan boleh lebih tinggi daripada keuntungan kewangan rumah maka saya tidak akan mencari dalam akaun saya akan mencari ke dalam ethereum atau beberapa crypto punks bergantung pada faktor risiko anda kerana rumah kajang dihantar tidak mengganggu dsr saya sejauh ini jadi sekarang apa posisi saya untuk mendapatkan kewangan ketiga jadi mungkin setelah melihat sekeliling saya menemui seorang penjual yang dapat menutupi perhubungan tetapi bank hanya menerima 70% dari itu saya boleh membayar perbezaan dengan menggunakan wang ini kerana sekali lagi ia menyusahkan saya dan aset tetapi jika anda tidak selesa dengan mencari')

In [9]:
# c = texts[0][1]
# prompt = f'{c}\n\ngenerate questions based on context above'
# formatted_prompt = format_prompt(prompt, [])
# stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
# output = stream.generated_text

In [10]:
prompts = []
for c in texts:
    prompt = (c[0], c[1], f'{c[1]}\n\ngenerate questions based on context above')
    prompts.append(prompt)
    
len(prompts)

514126

In [11]:
def answer(q, i):
    filename = f'mixtral-audio-instruction/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
        try:
            prompt = q[2]
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump((q[0], q[1], output), fopen)
            break
        except Exception as e:
            print(e)
            pass

In [12]:
# max_worker = 250

# questions = prompts
# for i in tqdm(range(0, len(questions), max_worker)):
#     urls_ = [(q, no + i) for no, q in enumerate(questions[i: i + max_worker])]
    
#     with ThreadPoolExecutor(max_workers=max_worker) as executor:
#         futures = {executor.submit(answer, url[0], url[1]): url for url in urls_}

#         for future in as_completed(futures):
#             future.result()

In [13]:
from glob import glob
import json

files = sorted(glob('mixtral-audio-instruction/*.json'))
len(files)

248499

In [15]:
from tqdm import tqdm

questions = []
for f in tqdm(files):
    index = int(os.path.split(f)[1].replace('.json', ''))
    filename = texts[index][0]
    with open(f) as fopen:
        data = json.load(fopen)
    if len(data) == 3:
        ind = 1
    else:
        ind = 0
    splitted = data[-1].strip().split('\n')
    splitted = [s for s in splitted if '.' if s and '?' in s]
    splitted = ['.'.join(s.split('.')[1:]).strip() for s in splitted]
    splitted = [s for s in splitted if len(s) > 3]
    splitted = [s[1:] if s[0] == '"' else s for s in splitted]
    splitted = [s[:-1] if s[-1] == '"' else s for s in splitted]
    splitted = splitted[:5]
    splitted = [(filename, data[ind], s.strip()) for s in splitted]
    questions.extend(splitted)
    
questions = [s for s in questions if len(s[2]) > 20]
len(questions)

In [ ]:
questions[0]

In [16]:
!mkdir answer-mixtral-audio-instruction
# !rm answer-mixtral-audio-instruction/*.json

mkdir: cannot create directory ‘answer-mixtral-audio-instruction’: File exists


In [17]:
def answer(q, i):
    filename = f'answer-mixtral-audio-instruction/{i}.json'
    if os.path.exists(filename):
        return
    
    question = f'{q[1]}\n{q[2]}'
    formatted_prompt = format_prompt(question, [])
    while True:
        try:
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            answer = stream.generated_text.strip()
            break
        except Exception as e:
            # print(e)
            pass 
    
    history = [(question, answer)]

    for _ in range(random.randint(1, 2)):
        formatted_prompt = format_user(history)
        
        while True:
            try:
                stream = client.text_generation(formatted_prompt, **generate_kwargs, stop_sequences = ['\n', '?', '[/INST]'], stream=False, details=True, return_full_text=False)
                u = stream.generated_text.strip()
                break
            except Exception as e:
                # print(e)
                pass
        
        formatted_prompt = format_prompt(u, history)
        while True:
            try:
                stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
                a = stream.generated_text.strip()
                break
            except Exception as e:
                # print(e)
                pass
            
        history.append((u, a))
        
    d = {
        'filename': q[0],
        'context': q[1],
        'question': q[2],
        'answer': answer,
        'multiturn': history
    }
    
    with open(filename, 'w') as fopen:
        json.dump(d, fopen)

In [18]:
answer(questions[10000], 10000)

In [20]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [21]:
from threading import Thread
from queue import Queue

queue = Queue()
urls = [(q, no) for no, q in enumerate(questions)]
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [ ]:
max_worker = 500
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

 24%|██████▊                     | 294193/1217727 [5:31:52<205:09:46,  1.25it/s]

In [ ]:
for i in range(len(consumers)):
    consumers[i].join()

In [3]:
from glob import glob

files = sorted(glob('answer-mixtral-audio-instruction/*.json'))
len(files)

293753

In [24]:
all_texts = []
for f in tqdm(files):
    try:
        with open(f) as fopen:
            data = json.load(fopen)
    except:
        continue
        
    all_texts.append(data['question'])
    all_texts.append(data['answer'])
        
    for d in data['multiturn'][1:]:
        all_texts.extend(d)
    
len(all_texts)

100%|████████████████████████████████| 293753/293753 [00:16<00:00, 18145.61it/s]


1483960

In [25]:
with open('mixtral-audio-instruction.texts', 'w') as fopen:
    for t in set(all_texts):
        if not len(t):
            continue
        fopen.write(f'{json.dumps(t)}\n')

In [26]:
!wc -l mixtral-audio-instruction.texts

1469270 mixtral-audio-instruction.texts


In [27]:
!cp mixtral-audio-instruction.texts ../translation

In [4]:
import re

mapping = {}
for f in glob('/home/husein/ssd3/translation/mixtral-audio-instruction.texts*.splitted.requested'):
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if 'Source text\nclear\nLook up details' in l['r']['result']:
                continue
            if 'clear\nLook up details' in l['r']['result']:
                continue
            if l['r']['result'].startswith('Source text\n'):
                continue

            n = l['r']['result']
            hypens = re.findall('\w+ -\w+', n)
            for h in hypens:
                splitted = h.split('-')
                if len(splitted) != 2:
                    continue
                splitted = [s.strip() for s in splitted]
                splitted = '-'.join(splitted)
                n = n.replace(h, splitted)
            mapping[l['src']] = n
            
len(mapping)

1469242

In [19]:
with open('mixtral-audio-instruction.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
            
        chat = [
            {'role': 'user', 'content': data['question'], 'content_ms': mapping.get(data['question'])},
            {'role': 'assistant', 'content': data['answer'], 'content_ms': mapping.get(data['answer'])}
        ]
        for d in data['multiturn'][1:]:
            chat.extend([
                {'role': 'user', 'content': d[0], 'content_ms': mapping.get(d[0])},
                {'role': 'assistant', 'content': d[1], 'content_ms': mapping.get(d[1])}
            ])

        d = {
            'context': data['context'],
            'chat': chat,
            'filename': data['filename']
        }
        fopen_l.write(f'{json.dumps(d)}\n')

100%|█████████████████████████████████████████████████████████████████████████████████████████| 293753/293753 [00:23<00:00, 12353.09it/s]


In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='mixtral-audio-instruction.jsonl',
    path_in_repo='mixtral-audio-instruction.jsonl',
    repo_id='mesolitica/malaysian-youtube-audio-instructions',
    repo_type='dataset',
)

mixtral-audio-instruction.jsonl:   0%|          | 0.00/2.01G [00:00<?, ?B/s]